In [1]:
from pyspark.sql import SparkSession
from PyClasses.Transformation import DE_Transformation
# Run Transformation
spark = SparkSession.builder.appName("DE-prj").getOrCreate()
try:
    transformation = DE_Transformation(spark)
    cleanned_sentences, words = transformation.start_transform_rawdata()
finally:
    spark.stop()

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/13 19:30:57 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/13 19:30:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/13 19:30:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java